In [1]:
import at
import sys
if sys.version_info.minor < 9:
    from importlib_resources import files, as_file
else:
    from importlib.resources import files, as_file

In [2]:
fname = 'hmba.mat'
with as_file(files('machine_data') / fname) as path:
    ring = at.load_lattice(path)

In [3]:
from at.future import ElementVariable, RefptsVariable

# Variables

Variables are **references** to any scalar quantity. Predefined classes are available
for accessing any scalar attribute of an element, or any item of an array attribute.

Any other quantity may be accessed by either subclassing the {py:class}`~.variables.Variable`
abstract base class, or using a {py:class}`~.variables.CustomVariable`.

## {py:class}`~.element_variables.ElementVariable`

An {py:class}`~.element_variables.ElementVariable` refers to a single attribute (or item of an array attribute) of one or several {py:class}`.Element` objects.

We now create a variable pointing to the length of all QF1 magnets of *ring*:

In [4]:
lf1 = ElementVariable(ring["QF1[AE]"], "Length", name="lf1")
print(f"lf1: {lf1}")
print(lf1.value)

lf1: ElementVariable(0.311896, name='lf1')
0.311896


and another variable pointing to the strength of the same magnets:

In [5]:
kf1 = ElementVariable(ring["QF1[AE]"], "PolynomB", index=1, name="kf1")
print("kf1:", kf1)
print(kf1.value)

kf1: ElementVariable(2.5394599781303304, name='kf1')
2.5394599781303304


We can check which elements are concerned by the `kf1` variable. The element container is a set, so that no element may appear twice:

In [6]:
kf1.elements

{Quadrupole('QF1A', 0.311896, 2.5394599781303304, FringeQuadEntrance=1, FringeQuadExit=1, NumIntSteps=20),
 Quadrupole('QF1E', 0.311896, 2.5394599781303304, FringeQuadEntrance=1, FringeQuadExit=1, NumIntSteps=20)}

`kf1` drives 2 quadrupoles. Let's look at the 1{sup}`st` one:

In [7]:
print(ring[5])

Quadrupole:
	FamName : QF1A
	Length : 0.311896
	PassMethod : StrMPoleSymplectic4Pass
	NumIntSteps : 20
	FringeQuadEntrance : 1
	FringeQuadExit : 1
	MaxOrder : 1
	PolynomA : [0. 0.]
	PolynomB : [0.         2.53945998]
	K : 2.5394599781303304


We can now change the strength of both QF1 magnets and check again the 1{sup}`st` one:

In [8]:
kf1.set(2.5)
print(ring[5])

Quadrupole:
	FamName : QF1A
	Length : 0.311896
	PassMethod : StrMPoleSymplectic4Pass
	NumIntSteps : 20
	FringeQuadEntrance : 1
	FringeQuadExit : 1
	MaxOrder : 1
	PolynomA : [0. 0.]
	PolynomB : [0.  2.5]
	K : 2.5


We can look at the history of `kf1` values

In [9]:
kf1.history

[2.5394599781303304, 2.5]

And revert to the initial or previous values:

In [10]:
kf1.set_previous()
print(ring[5])

Quadrupole:
	FamName : QF1A
	Length : 0.311896
	PassMethod : StrMPoleSymplectic4Pass
	NumIntSteps : 20
	FringeQuadEntrance : 1
	FringeQuadExit : 1
	MaxOrder : 1
	PolynomA : [0. 0.]
	PolynomB : [0.         2.53945998]
	K : 2.5394599781303304


An {py:class}`~.element_variables.ElementVariable` is linked to Elements. It will not follow any copy of the element, neither shallow nor deep. So if we make a copy of ring:

In [11]:
newring = ring.deepcopy()
print(f"ring:   {ring[5].PolynomB[1]}")
print(f"newring: {newring[5].PolynomB[1]}")

ring:   2.5394599781303304
newring: 2.5394599781303304


and modify the `kf1` variable:

In [12]:
kf1.set(2.6)

In [13]:
print(f"ring:   {ring[5].PolynomB[1]}")
print(f"newring: {newring[5].PolynomB[1]}")

ring:   2.6
newring: 2.5394599781303304


The QF1 in newring is not affected.

One can set upper and lower bounds on a variable. Trying to set a value out of the bounds will raise a {py:obj}`ValueError`. The default is (-{py:obj}`numpy.inf`, {py:obj}`numpy.inf`).

In [14]:
lfbound = ElementVariable(ring["QF1[AE]"], "Length", bounds=(0.30, 0.35))

In [15]:
lfbound.set(0.2)

ValueError: set value must be in (0.3, 0.35)

Variables also accept a *step* keyword argument. Its value is used as the initial step in matching, and in the {py:meth}`~.variables.Variable.step_up` and {py:meth}`~.variables.Variable.step_down` methods.

## {py:class}`.RefptsVariable`

An {py:class}`.RefptsVariable` is similar to an {py:class}`~.element_variables.ElementVariable` but it is not associated with an {py:class}`~.Element`
itself, but with its location in a Lattice. So it will act on any lattice with the same elements.

But it needs a *ring* keyword in its *set* and *get* methods, to identify the selected lattice.

In [16]:
kf2 = RefptsVariable("QF1[AE]", "PolynomB", index=1, name="kf2")

We can now use this variable on the two rings:

In [17]:
kf2.set(2.55, ring=ring)
kf2.set(2.45, ring=newring)

In [18]:
print(f"ring:   {ring[5].PolynomB[1]}")
print(f"newring: {newring[5].PolynomB[1]}")

ring:   2.55
newring: 2.45


## Custom variables